In [7]:
from transformers import VivitConfig

from model.vivit import VivitPose
from preprocess_data import get_video_datasets
from torch.utils.data import DataLoader
import torch
device = 'cuda'
configuration = VivitConfig()
vivit_num_frames = 5
configuration.num_labels = 201
configuration.num_frames=vivit_num_frames
model = VivitPose(configuration).to(device)
train_data, test_data = get_video_datasets()
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)


In [8]:
#clear gc, cuda cache
import gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
history = []
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
model.train()
for i, v in enumerate(train_loader):
    #sliding window of 32 frames or less, if we are at the beginning
    
    all_frames = v[0]/255.0
    num_frames = all_frames.shape[1]
    labels = v[1]
    labels = labels.to(device)
    video_loss = 0
    # if len(v[0]) < 32:
    #     continue
    for j in range(vivit_num_frames-1, num_frames):
        start_range =  (j-(vivit_num_frames-1))
        if start_range < 0:
            break
        end_range = j+1
        # v is [batch, frames, height, width, channels]
        frames = all_frames[:,start_range:end_range].detach().clone()
        #convert frames to byte tensor
        frames = frames.to(device)
        frames = frames.permute(0,1,4,2,3)
        output = model(frames)
        label = labels[:,j]
        loss = criterion(output, label)
        loss.backward()
        video_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        history.append(loss.item())
        torch.cuda.empty_cache()
    print(video_loss/(num_frames-vivit_num_frames+1))


loss:  0.5907091498374939
loss:  0.3519069254398346
loss:  0.07536350935697556
loss:  0.04098983108997345
loss:  0.030397765338420868
loss:  0.026937702670693398
loss:  0.023042790591716766
loss:  0.018921643495559692
loss:  0.015516463667154312
loss:  0.012925740331411362
loss:  0.010789918713271618
loss:  0.008914338424801826
loss:  0.007304217666387558
loss:  0.006072200834751129
loss:  0.0052165924571454525
loss:  0.0046972911804914474
loss:  0.004374220035970211
loss:  0.004169862251728773
loss:  0.0039529637433588505
loss:  0.0037135060410946608
loss:  0.003436537692323327
loss:  0.003081295173615217
loss:  0.0027440746780484915
loss:  0.0023700143210589886
loss:  0.002003422938287258
loss:  0.001676481100730598
loss:  0.001392290578223765
loss:  0.0012092656688764691
loss:  0.0011020827805623412
loss:  0.0010656032245606184
loss:  0.00105462153442204
loss:  0.0010230487678200006
loss:  0.000992905697785318
loss:  0.000930040783714503
loss:  0.000839955871924758
loss:  0.00073765

KeyboardInterrupt: 